In [1]:
import sys

import matplotlib.pyplot as plt
import numpy as np

sys.path.append("/home/uttam/Dropbox/Projects/timeseries_generator/src/")
from generators import Particle

In [2]:
dimension = 2
particle = Particle(location=np.zeros(dimension), fertility=0.1, mortality=0.05)
particles_dict = {}
particles_dict[0] = particle
particle_id_max = 0

In [3]:
len(particles_dict)

1

In [4]:
for i in range(1000):
    new_particles_list = []
    killed_particles_list = []
    for particle_id in particles_dict.keys():
        particle = particles_dict[particle_id]
        particle.move(dtime=1.0)
        reproduce = particle.reproduce(dtime=1.0)
        if reproduce:
            new_particles_list.append(reproduce)
        survive = particle.survive(dtime=1.0)
        if not survive:
            killed_particles_list.append(particle_id)
    if len(new_particles_list) > 0:
        for i, particle in enumerate(new_particles_list):
            particles_dict[particle_id_max + i + 1] = particle
        particle_id_max = particle_id_max + len(new_particles_list)
    if len(killed_particles_list) > 0:
        for particle_id in killed_particles_list:
            del particles_dict[particle_id]
    print(len(particles_dict))


1
2
2
2
2
3
3
5
5
5
5
4
4
4
5
5
7
7
8
8
10
11
13
14
15
15
18
19
22
22
23
24
24
27
29
26
27
29
30
31
31
34
36
37
41
44
44
43
43
45
48
46
51
52
58
62
65
67
75
78
74
82
86
96
107
120
126
130
133
143
151
160
173
180
184
194
203
216
227
233
240
247
256
270
284
307
323
325
337
356
376
397
427
462
501
518
558
591
616
640
665
697
744
777
827
866
917
960
995
1053
1095
1159
1236
1279
1333
1387
1474
1556
1620
1701
1769
1848
1933
2027
2133
2253
2357
2500
2632
2776
2991
3129
3283
3442
3618
3824
4066
4276
4496
4710
4977
5218
5472
5746
6012
6239
6561
6905
7258
7634


KeyboardInterrupt: 

In [ ]:
plt.scatter(np.array(particle_locations_list)[:1000, 0], np.array(particle_locations_list)[:1000, 1])
plt.show()